In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df = df.dropna()

In [6]:
X = df.drop('label', axis=1)
y = df['label']

In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'2.14.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
voc_size = 5000

In [13]:
messages = X.copy()

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/lenovo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[1753, 2197, 3178, 2794, 3533, 2558, 1288, 4578, 258, 1387],
 [3173, 3275, 3059, 3186, 3359, 2523, 4222],
 [676, 1602, 2535, 2810],
 [2963, 773, 2053, 3327, 566, 4557],
 [3332, 3359, 582, 2799, 3121, 2214, 3359, 228, 69, 3033],
 [2373,
  2815,
  118,
  3349,
  3925,
  4507,
  1805,
  36,
  1600,
  1143,
  3460,
  4887,
  4144,
  4712,
  4222],
 [456, 1989, 3144, 1375, 3033, 4939, 4179, 3979, 15, 1281, 342],
 [397, 3075, 1754, 2383, 2785, 1739, 4507, 3915, 15, 1281, 342],
 [4104, 2798, 3616, 897, 135, 4930, 1751, 4182, 4507, 4694],
 [1274, 2193, 4548, 2180, 3000, 1345, 4029, 3816],
 [2452, 3207, 1672, 2913, 3606, 576, 4020, 767, 4781, 4196, 234],
 [3327, 788, 3533, 4930, 4507, 2785],
 [4904, 2458, 273, 1797, 2614, 3616, 891, 3608, 1540],
 [1078, 3950, 1930, 459, 1043, 3344, 2789, 15, 1281, 342],
 [2212, 1315, 787, 4764, 4043, 15, 1281, 342],
 [3156, 2085, 2051, 840, 1627, 4643, 4487, 785, 593, 2368],
 [2155, 2579, 3275],
 [539, 4165, 2431, 3840, 4507, 3240, 4811, 4222],
 [1114, 745, 30

In [22]:
# pad_sequence fixed all sentences to 20
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [23]:
print(embedded_docs)

[[   0    0    0 ... 4578  258 1387]
 [   0    0    0 ... 3359 2523 4222]
 [   0    0    0 ... 1602 2535 2810]
 ...
 [   0    0    0 ...   15 1281  342]
 [   0    0    0 ... 4048 1974  611]
 [   0    0    0 ... 4853 2149 4451]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1753,
       2197, 3178, 2794, 3533, 2558, 1288, 4578,  258, 1387], dtype=int32)

In [31]:
# creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [32]:
import numpy as np

In [33]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [35]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 19ms/step - loss: 0.3323 - accuracy: 0.8449 - val_loss: 0.1914 - val_accuracy: 0.9196
Epoch 2/10
192/192 [==============================] - 3s 17ms/step - loss: 0.1432 - accuracy: 0.9435 - val_loss: 0.1909 - val_accuracy: 0.9191
Epoch 3/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0987 - accuracy: 0.9624 - val_loss: 0.2265 - val_accuracy: 0.9223
Epoch 4/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0713 - accuracy: 0.9745 - val_loss: 0.2412 - val_accuracy: 0.9165
Epoch 5/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0494 - accuracy: 0.9833 - val_loss: 0.2769 - val_accuracy: 0.9178
Epoch 6/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0351 - accuracy: 0.9894 - val_loss: 0.3225 - val_accuracy: 0.9167
Epoch 7/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0206 - accuracy: 0.9935 - val_loss: 0.3605 - val_accuracy: 0.9095